
Suppose we have a function $g(x)$. Prover wants to prove that he knows the right values for the set of inputs $arr$

In [1]:
import math
import numpy as np
from scipy.stats import norm

In [2]:
arr = [x for x in range(100, 200)]
len(arr)

100

In [3]:
def g(x):
    return  math.exp(x / 2) / x**10

In [4]:
results = [g(x) for x in arr]
# print(results)

Now we have a (x, g(x)) results. We then easily encode it into a polynomial.

### STARK for Black-Scholes

In [5]:

N = norm.cdf

def BS_CALL(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return S * N(d1) - K * np.exp(-r*T)* N(d2)

def BS_PUT(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    d2 = d1 - sigma* np.sqrt(T)
    return K*np.exp(-r*T)*N(-d2) - S*N(-d1)   

In [6]:
# BS call, put delta

def delta_BS_put(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    return N(d1) - 1

def delta_BS_call(S, K, T, r, sigma):
    d1 = (np.log(S/K) + (r + sigma**2/2)*T) / (sigma*np.sqrt(T))
    return N(d1)

Suppose we have a $V$ arbitrary positions. $|V| = 100$, which means that amount of positions equals 100.
 $\\$
Each position equals to a vector $\vec{v} = 
\begin{pmatrix}
S \\ K \\ T-t \\ vol \\ type
\end{pmatrix}
$
where $type$ is either $call$ or $put$

#### STARK generation
1. Prover computes $BS\_delta(\vec{v})$ for every $\vec{v}\in V$. (100 single computations)
2. Prover generates evenly distributed vectors $\vec{v'}$ in amount of $100*10^6 = 10^8$, and computes $BS\_delta(\vec{v'})$ for every generated point
3. To summarize steps 1-2, now prover computed deltas for every of $N = (100 + 10^8)$ points
4. Then prover encodes all these pairs $(\vec{v}, \: BS\_delta(\vec{v}))$ into a polynomial $P$ with a degree $N$
5. Prover commits this polynomial and the array of 100 computed points (that are related to real positions)

#### Verification
1. Verificator choses a random $\vec{v}_{ver}$ point of $100+10^8$ points, compares  $P(\vec{v}_{ver})\:?=\:BS\_delta(\vec{v}_{ver})$ $\\$
&nbsp; a) 


Some other thoughts...

We need to compute 100 deltas off-chain. We randomly generate another $10^8$ positions and provide a merkle root of this tree.
Verificator randomly choose 4 (of $10^8$) positions and requests for providing a merkle proof and result of computations delta in this specified position. Thus, the prover's answer is (merkle_path, unhashed leaf (i.e. the delta)). 

The probability of acceptance is... 

that not works. because prover may incorrectly compute in that 100 points but correctly compute in other $10^8$ points


Ok, what if we compute in points and encode it into a polynomial of a degree 100. Then we 